In [1]:
from py2neo import Graph
import pandas as pd
import sys
import numpy as np
import boto3
import psycopg2
import db_utils
import tqdm
import matplotlib

Strategy based on such article: https://medium.com/@crimy/one-shot-learning-siamese-networks-and-triplet-loss-with-keras-2885ed022352

# 1. Extract data of loaded images on S3

In [2]:
def query_results_to_df(query_results):
    if len(query_results) == 0:
        return False
    cols = ["instance_id","stat","track_id","win","ini","fin","rows","cols","date"]
    return pd.DataFrame(query_results, columns=cols)

In [3]:
query_status = """ 
SELECT * FROM status_specto
"""
ENDPOINT="tracksurl.czjs6btlvfgd.eu-west-2.rds.amazonaws.com"
PORT="5432"
USR="david"
REGION="eu-west-2"
DBNAME="postgres"
PSSWD=["qrks","jfut","iv","uf","1"]

conn = psycopg2.connect(host=ENDPOINT, port=PORT, database=DBNAME, user=USR, password=''.join(PSSWD))
cur = conn.cursor()
cur.execute(query_status)
query_results = cur.fetchall()
df_status = query_results_to_df(query_results)
conn.close()

## 1.1 Filter

### 1.1.1 Images that are not well-shaped

In [4]:
df = df_status[df_status["cols"].isin({937, 938})]

In [5]:
df.head()

instance_id  stat                track_id  win  ini  fin  rows  \
0  i-046cdfe3763e0b089     1  000xQL6tZNLJzIrtIgxqSl    0    0   30   256   
1  i-046cdfe3763e0b089     1  000xQL6tZNLJzIrtIgxqSl    1   20   50   256   
2  i-046cdfe3763e0b089     1  000xQL6tZNLJzIrtIgxqSl    3   60   90   256   
3  i-046cdfe3763e0b089     1  000xQL6tZNLJzIrtIgxqSl    2   40   70   256   
4  i-046cdfe3763e0b089     1  000xQL6tZNLJzIrtIgxqSl    4   80  110   256   

   cols                date  
0   937 2020-07-05 21:47:21  
1   937 2020-07-05 21:47:21  
2   937 2020-07-05 21:47:21  
3   937 2020-07-05 21:47:21  
4   937 2020-07-05 21:47:21

In [6]:
df.shape

(644047, 9)

In [7]:
set_tracks = set(df.track_id)

In [8]:
print(len(set_tracks))

56222


# 2. Dataset Creation (train/test) and Label (based on Neo4j Graph)

In [10]:
import db_utils

In [17]:
conn = psycopg2.connect("dbname=spotify")
cursor= conn.cursor()

In [27]:
# Neo4j
graph = Graph(bolt = True, host = "localhost", name = "Spotify", user = "neo4j", password = "qrks")

## 2.1 Path 1) Songs from top genres

This is like a PoC, we will:
- take the most popular genres, top 5 genres, drop the undefined genre
- look for the artists for each genre
- get all the songs for that artist
- check if they are in our database
- do triplets of songs: select one anchor ( song A1 from artist A), then select as the postive pair of the triplet another song from that artist (A2). Then choose one song from a related artist of A (i.e artist B). Take as the negative pair of the triplet the B1 song for example. Hence the triplet will be formet by A1-A2-B1. This poses a much more difficult task since related artists will tend to share features in their songs so that will be more challenging rather than taking as the negative pair any random song.

### a) SELECT top genres

In [13]:
query = """
SELECT * FROM (
select genre, count(distinct artist_id) as num_art from master_genre GROUP BY genre) x1
WHERE num_art > 100
ORDER BY num_art DESC;
"""

In [18]:
df_genres = db_utils.select_table(cursor, direct_query=query,
                                 sel_cols=["track_id", "num_art"])
df_genres.drop_duplicates(inplace=True)
conn.close()

In [23]:
# Remove the undefined genre
df_genres = df_genres[df_genres["track_id"] != "undefined"]

# Sort genres by the number of artists
df_genres.sort_values("num_art", ascending = False)

# Take the top 5
top_genres = df_genres.head(5)

In [24]:
top_genres

track_id  num_art
1             pop      479
2       dance pop      393
3           latin      264
4             rap      239
5  tropical house      233

In [25]:
cypher_query_triplets = """


MATCH (g1:Genre)-[:GEN_ART]->(a1:Artist)-[:REL_ART]->(a2:Artist)<-[:GEN_ART]-(g2:Genre)
WHERE g1.genre_id IN ["pop", "dance pop", "latin", "rap", "tropical house"]
    AND g2.genre_id IN ["pop", "dance pop", "latin", "rap", "tropical house"]
WITH a1, a2, g1, g2
MATCH  (t1:Track)<-[:ART_TR]-(a1:Artist),
       (t2:Track)<-[:ART_TR]-(a1:Artist),
       (t3:Track)<-[:ART_TR]-(a2:Artist)

RETURN t1.track_id, t2.track_id, a1.artist_id, t3.track_id, a2.artist_id, t1.track_name, t2.track_name,
        a1.artist_name, t3.track_name, a2.artist_name, g1.genre_id, g2.genre_id

"""


In [28]:
cursor_neo4j = graph.run(cypher_query_triplets)

In [26]:
df_triplets = pd.DataFrame.from_records(cursor_neo4j, columns=cursor_neo4j.keys())

AttributeError: 'psycopg2.extensions.cursor' object has no attribute 'keys'

## 2.2 Path 2) Artist sharing